In [2]:
%pip install -r requirements.txt

  Using cached cmake-3.27.7-py2.py3-none-win_amd64.whl.metadata (6.8 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached jsonpickle-3.0.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached pickle5-0.0.11.tar.gz (132 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using cached jsonpickle-3.0.2-py3-none-any.whl (40 kB)
Using cached cmake-3.27.7-py2.py3-none-win_amd64.whl (34.6 MB)
  Running setup.py clean for pickle5
Failed to build pickle5
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [40 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\pickle5
  copying pickle5\pickle.py -> build\lib.win-amd64-cpython-311\pickle5
  copying pickle5\pickletools.py -> build\lib.win-amd64-cpython-311\pickle5
  copying pickle5\__init__.py -> build\lib.win-amd64-cpython-311\pickle5
  creating build\lib.win-amd64-cpython-311\pickle5\test
  copying pickle5\test\pickletester.py -> build\lib.win-amd64-cpython-311\pickle5\test
  copying pickle5\test\test_pickle.py -> build\lib.win-amd64-cpython-311\pickle5\test
  copying pickle5\test\test_picklebuffer.py -> build\lib.win-amd64-cpython-311\pickle5\test
  copying pickle5\test\__init__.py -> build\lib.win-amd64-cpython-311\pickle5\test
  running build_ext
  building 'pickle5._pickle' extension
  creating 

## Importing Packages

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from itertools import product
from datetime import timedelta

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

## Loading Dataset

In [9]:
df = pd.read_csv('DataFrames/final_merged_copper_dataset.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df["Date"] = pd.to_datetime(df["Date"])
df

,Date,Price,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,2000-01-04,0.8480,1.0,2000.0,14096.033,168.8,3.376857,59.470000
1,2000-01-05,0.8565,1.0,2000.0,14096.033,168.8,3.376857,59.470000
2,2000-01-06,0.8530,1.0,2000.0,14096.033,168.8,3.376857,59.470000
3,2000-01-07,0.8540,1.0,2000.0,14096.033,168.8,3.376857,59.470000
4,2000-01-10,0.8465,1.0,2000.0,14096.033,168.8,3.376857,59.470000
...,...,...,...,...,...,...,...,...
6079,2023-11-24,3.8205,11.0,2023.0,19374.000,307.0,6.600000,168.657489
6080,2023-11-27,3.7890,11.0,2023.0,19374.000,307.0,6.600000,168.657489
6081,2023-11-28,3.8285,11.0,2023.0,19374.000,307.0,6.600000,168.657489
6082,2023-11-29,3.8255,11.0,2023.0,19374.000,307.0,6.600000,168.657489


In [10]:
last_date = df['Date'].max().date()
last_date

datetime.date(2023, 11, 30)

In [11]:
df

,Date,Price,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,2000-01-04,0.8480,1.0,2000.0,14096.033,168.8,3.376857,59.470000
1,2000-01-05,0.8565,1.0,2000.0,14096.033,168.8,3.376857,59.470000
2,2000-01-06,0.8530,1.0,2000.0,14096.033,168.8,3.376857,59.470000
3,2000-01-07,0.8540,1.0,2000.0,14096.033,168.8,3.376857,59.470000
4,2000-01-10,0.8465,1.0,2000.0,14096.033,168.8,3.376857,59.470000
...,...,...,...,...,...,...,...,...
6079,2023-11-24,3.8205,11.0,2023.0,19374.000,307.0,6.600000,168.657489
6080,2023-11-27,3.7890,11.0,2023.0,19374.000,307.0,6.600000,168.657489
6081,2023-11-28,3.8285,11.0,2023.0,19374.000,307.0,6.600000,168.657489
6082,2023-11-29,3.8255,11.0,2023.0,19374.000,307.0,6.600000,168.657489


In [14]:
LOOKBACK = 15
data = pd.DataFrame({'Date': df["Date"].copy(),
                     'Price': df["Price"].copy()})
# Select the "Date" column and store it in the variable 'train_dates'.
dates = data["Date"]

# Exclude the first 'LOOKBACK' dates from 'train_dates' to create a time window.
dates = dates[LOOKBACK:]
dates

15     2000-01-26
16     2000-01-27
17     2000-01-28
18     2000-01-31
19     2000-02-01
          ...    
6079   2023-11-24
6080   2023-11-27
6081   2023-11-28
6082   2023-11-29
6083   2023-11-30
Name: Date, Length: 6069, dtype: datetime64[ns]

In [15]:
data

,Date,Price
0,2000-01-04,0.8480
1,2000-01-05,0.8565
2,2000-01-06,0.8530
3,2000-01-07,0.8540
4,2000-01-10,0.8465
...,...,...
6079,2023-11-24,3.8205
6080,2023-11-27,3.7890
6081,2023-11-28,3.8285
6082,2023-11-29,3.8255


In [16]:
dates.reset_index(drop=True, inplace=True)

In [17]:
dates

0      2000-01-26
1      2000-01-27
2      2000-01-28
3      2000-01-31
4      2000-02-01
          ...    
6064   2023-11-24
6065   2023-11-27
6066   2023-11-28
6067   2023-11-29
6068   2023-11-30
Name: Date, Length: 6069, dtype: datetime64[ns]

In [18]:
from copy import deepcopy as dc


def prepare_dataframe_for_lstm(df, n_steps):
    df = dc(df)

    #df.set_index('Date', inplace=True)

    for i in range(1, n_steps + 1):
        df[f'Price(t-{i})'] = df['Price'].shift(i)

    df.dropna(inplace=True)

    return df


shifted_df = prepare_dataframe_for_lstm(data, LOOKBACK)
shifted_df = shifted_df.reset_index(drop=True)
df = df.drop(df.index[-7:])
df

,Date,Price,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,2000-01-04,0.8480,1.0,2000.0,14096.033,168.8,3.376857,59.470000
1,2000-01-05,0.8565,1.0,2000.0,14096.033,168.8,3.376857,59.470000
2,2000-01-06,0.8530,1.0,2000.0,14096.033,168.8,3.376857,59.470000
3,2000-01-07,0.8540,1.0,2000.0,14096.033,168.8,3.376857,59.470000
4,2000-01-10,0.8465,1.0,2000.0,14096.033,168.8,3.376857,59.470000
...,...,...,...,...,...,...,...,...
6072,2023-11-15,3.7185,11.0,2023.0,19374.000,307.0,6.600000,168.657489
6073,2023-11-16,3.7025,11.0,2023.0,19374.000,307.0,6.600000,168.657489
6074,2023-11-17,3.7345,11.0,2023.0,19374.000,307.0,6.600000,168.657489
6075,2023-11-20,3.8335,11.0,2023.0,19374.000,307.0,6.600000,168.657489


In [19]:
shifted_df

,Date,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7),Price(t-8),Price(t-9),Price(t-10),Price(t-11),Price(t-12),Price(t-13),Price(t-14),Price(t-15)
0,2000-01-26,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,0.8565,0.8480
1,2000-01-27,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,0.8565
2,2000-01-28,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530
3,2000-01-31,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540
4,2000-02-01,0.8480,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6064,2023-11-24,3.8205,3.7743,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,3.6790,3.7190,3.6815
6065,2023-11-27,3.7890,3.8205,3.7743,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,3.6790,3.7190
6066,2023-11-28,3.8285,3.7890,3.8205,3.7743,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,3.6790
6067,2023-11-29,3.8255,3.8285,3.7890,3.8205,3.7743,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380


In [20]:
shifted_df["Real_GDP"] = df["Real_GDP"]
shifted_df["CPI"] = df["CPI"]
shifted_df["inflation_rate"] = df["inflation_rate"]
shifted_df["PALLFNFINDEXM"] = df["PALLFNFINDEXM"]
shifted_df

,Date,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7),Price(t-8),...,Price(t-10),Price(t-11),Price(t-12),Price(t-13),Price(t-14),Price(t-15),Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,2000-01-26,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,...,0.8435,0.8465,0.8540,0.8530,0.8565,0.8480,14096.033,168.8,3.376857,59.470000
1,2000-01-27,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,...,0.8660,0.8435,0.8465,0.8540,0.8530,0.8565,14096.033,168.8,3.376857,59.470000
2,2000-01-28,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,...,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,14096.033,168.8,3.376857,59.470000
3,2000-01-31,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,...,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,14096.033,168.8,3.376857,59.470000
4,2000-02-01,0.8480,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,...,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,14096.033,168.8,3.376857,59.470000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6064,2023-11-24,3.8205,3.7743,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,...,3.5870,3.6405,3.6380,3.6790,3.7190,3.6815,19374.000,307.0,6.600000,168.657489
6065,2023-11-27,3.7890,3.8205,3.7743,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,...,3.6660,3.5870,3.6405,3.6380,3.6790,3.7190,19374.000,307.0,6.600000,168.657489
6066,2023-11-28,3.8285,3.7890,3.8205,3.7743,3.7970,3.8395,3.8335,3.7345,3.7025,...,3.6835,3.6660,3.5870,3.6405,3.6380,3.6790,19374.000,307.0,6.600000,168.657489
6067,2023-11-29,3.8255,3.8285,3.7890,3.8205,3.7743,3.7970,3.8395,3.8335,3.7345,...,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,19374.000,307.0,6.600000,168.657489


In [24]:
price_columns = [col for col in shifted_df.columns if 'Price' in col]
price_columns

['Price',
 'Price(t-1)',
 'Price(t-2)',
 'Price(t-3)',
 'Price(t-4)',
 'Price(t-5)',
 'Price(t-6)',
 'Price(t-7)',
 'Price(t-8)',
 'Price(t-9)',
 'Price(t-10)',
 'Price(t-11)',
 'Price(t-12)',
 'Price(t-13)',
 'Price(t-14)',
 'Price(t-15)']

In [26]:
prices_before_scaling_as_np = shifted_df[price_columns].values
prices_before_scaling_as_np

array([[0.8415, 0.8425, 0.8565, ..., 0.853 , 0.8565, 0.848 ],
       [0.8445, 0.8415, 0.8425, ..., 0.854 , 0.853 , 0.8565],
       [0.84  , 0.8445, 0.8415, ..., 0.8465, 0.854 , 0.853 ],
       ...,
       [3.8285, 3.789 , 3.8205, ..., 3.6405, 3.638 , 3.679 ],
       [3.8255, 3.8285, 3.789 , ..., 3.587 , 3.6405, 3.638 ],
       [3.8213, 3.8255, 3.8285, ..., 3.666 , 3.587 , 3.6405]])

In [27]:
prices_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_prices = prices_scaler.fit_transform(prices_before_scaling_as_np)
scaled_prices

array([[-0.89235389, -0.89189189, -0.88542389, ..., -0.88704089,
        -0.88542389, -0.88935089],
       [-0.89096789, -0.89235389, -0.89189189, ..., -0.88657889,
        -0.88704089, -0.88542389],
       [-0.89304689, -0.89096789, -0.89235389, ..., -0.89004389,
        -0.88657889, -0.88704089],
       ...,
       [ 0.48764149,  0.46939247,  0.48394548, ...,  0.4007854 ,
         0.3996304 ,  0.41857242],
       [ 0.48625549,  0.48764149,  0.46939247, ...,  0.37606838,
         0.4007854 ,  0.3996304 ],
       [ 0.48431508,  0.48625549,  0.48764149, ...,  0.41256641,
         0.37606838,  0.4007854 ]])

In [28]:
scaled_prices.shape

(6069, 16)

In [29]:
GDP_before_scaling_as_np = shifted_df["Real_GDP"].to_numpy()
CPI_before_scaling_as_np = shifted_df["CPI"].to_numpy()
inflation_rate_before_scaling_as_np = shifted_df["inflation_rate"].to_numpy()
PALLFNFINDEXM_before_scaling_as_np = shifted_df["PALLFNFINDEXM"].to_numpy()

In [30]:
GDP_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_GDP = GDP_scaler.fit_transform(GDP_before_scaling_as_np.reshape(-1, 1))

In [31]:
scaled_GDP

array([[-1.        ],
       [-1.        ],
       [-1.        ],
       ...,
       [ 0.36628638],
       [ 0.36628638],
       [ 0.36628638]])

In [35]:
CPI_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_CPI = CPI_scaler.fit_transform(CPI_before_scaling_as_np.reshape(-1, 1))

In [36]:
scaled_CPI

array([[-1.        ],
       [-1.        ],
       [-1.        ],
       ...,
       [ 0.98864658],
       [ 0.98864658],
       [ 0.98864658]])

In [37]:
IR_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_IR = IR_scaler.fit_transform(inflation_rate_before_scaling_as_np.reshape(-1, 1))

In [38]:
scaled_IR

array([[-0.10690381],
       [-0.10690381],
       [-0.10690381],
       ...,
       [ 0.66433555],
       [ 0.66433555],
       [ 0.66433555]])

In [39]:
PALLFNFINDEXM_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_PALLFNFINDEXM = PALLFNFINDEXM_scaler.fit_transform(PALLFNFINDEXM_before_scaling_as_np.reshape(-1, 1))

In [40]:
scaled_PALLFNFINDEXM

array([[-0.88354852],
       [-0.88354852],
       [-0.88354852],
       ...,
       [ 0.24367179],
       [ 0.24367179],
       [ 0.24367179]])

In [ ]:

stacked_np = np.hstack((scaled_prices, scaled_GDP,scaled_CPI,scaled_IR,scaled_PALLFNFINDEXM))

# Display the combined NumPy array
stacked_np.shape


In [ ]:
scaled_prices.shape

In [ ]:
scaled_df = shifted_df.drop(['Date'], axis=1)
scaled_df

In [ ]:
flipped_df = shifted_df[shifted_df.columns[::-1]]
flipped_df = flipped_df.drop(['Date'], axis=1)
flipped_df

In [ ]:
flipped_df.reset_index(drop=True, inplace=True)

In [ ]:
df.drop(columns=['Price'], inplace=True)

In [ ]:
new_df = pd.concat([df, flipped_df], axis=1)

In [ ]:
new_df

In [ ]:
new_df.drop(columns=['Month', 'Year'], inplace=True)
new_df

In [ ]:
flipped_df

In [ ]:
flipped_df_before_scaling_as_np = flipped_df.to_numpy()
flipped_df_before_scaling_as_np

In [ ]:
price_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_prices = price_scaler.fit_transform(flipped_df_before_scaling_as_np)

In [ ]:
scaled_prices

In [ ]:
result = np.concatenate((scaled_GDP, scaled_CPI, scaled_IR, scaled_PALLFNFINDEXM, scaled_prices), axis=1)
result

In [ ]:
X = result[:, 0:11]
y = result[:, 11]
X = X.astype(np.float32)
y = y.astype(np.float32)
X.dtype

In [ ]:
X

In [ ]:
split_index = int(len(X) * 0.95)
split_date = dates[split_index]

X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]
train_dates = dates[:split_index]
test_dates = dates[split_index:]
test_dates

In [ ]:
X_train

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.reshape((-1, LOOKBACK+4, 1))
X_test = X_test.reshape((-1, LOOKBACK+4, 1))

y_train = y_train.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))

In [ ]:
X_train = torch.tensor(X).float()
y_train = torch.tensor(y).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

#X_test.dtype

In [ ]:
X_test.shape

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

In [ ]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers,
                            batch_first=True)

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model = LSTM(1, 4, 1)
model.to(device)
model

In [ ]:
def train_one_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0

    for batch_index, batch in enumerate(train_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 100 == 99:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.5f}'.format(batch_index+1,
                                                    avg_loss_across_batches))
            running_loss = 0.0
    print()

In [ ]:
def validate_one_epoch():
    model.train(False)
    running_loss = 0.0

    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)

    print('Val Loss: {0:.5f}'.format(avg_loss_across_batches))
    print('***************************************************')
    print()

In [ ]:
learning_rate = 0.001
num_epochs = 30
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#Adam does the gradient descent

for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

In [ ]:
with torch.no_grad():
    predicted = model(X_train[:].to(device)).to('cpu').numpy()

plt.plot(train_dates, y_train, label='Actual Price')
plt.plot(train_dates, predicted, label='Predicted Price')

plt.xlabel('Dates')
plt.ylabel('Price')
plt.legend()
plt.show()


In [ ]:
train_predictions = predicted.flatten()

dummies = np.zeros((X_train.shape[0], LOOKBACK+1))
dummies[:, 0] = train_predictions
dummies = price_scaler.inverse_transform(dummies)

train_predictions = dc(dummies[:, 0])
train_predictions

In [ ]:
dummies = np.zeros((X_train.shape[0], LOOKBACK+1))
dummies[:, 0] = y_train.flatten()
dummies = price_scaler.inverse_transform(dummies)
new_y_train = dc(dummies[:, 0])
new_y_train

In [ ]:
plt.plot(train_dates,new_y_train,  label='Actual Price')
plt.plot(train_dates,train_predictions, label='Predicted Price')
plt.xlabel('Day')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
test_predictions = model(X_test.to(device)).detach().cpu().numpy().flatten()

dummies = np.zeros((X_test.shape[0], LOOKBACK+1))
dummies[:, 0] = test_predictions
dummies = price_scaler.inverse_transform(dummies)

test_predictions = dc(dummies[:, 0])
test_predictions

In [ ]:
dummies = np.zeros((X_test.shape[0], LOOKBACK+1))
dummies[:, 0] = y_test.flatten()
dummies = price_scaler.inverse_transform(dummies)

new_y_test = dc(dummies[:, 0])
new_y_test
squared_differences = (new_y_test - test_predictions)**2

# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(np.mean(squared_differences))
#rmse
absolute_differences = np.abs(new_y_test - test_predictions)

# Calculate the Mean Absolute Error (MAE)
mae = np.mean(absolute_differences)
rmse

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(test_dates, new_y_test, label='Actual Price')
plt.plot(test_dates,test_predictions, label='Predicted Price')
plt.xlabel('Day')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
test_dates

In [ ]:
"""# Instantiate your model
model = LSTM(1, 4, 1)

# Save the model to a .pkl (pickle) file
model_file_path = 'LSTM.pkl'
with open(model_file_path, 'wb') as file:
    pickle.dump(model, file)"""

In [ ]:
torch.save(model, 'model.pt')

In [ ]:
m1 = torch.load('model.pt')

In [ ]:
m1

In [ ]:
model

In [ ]:
start_date = last_date+timedelta(days=1)
end_date = "2024-01-25"
forecast_dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a DataFrame with the date column
forecast_set = pd.DataFrame({'Date': forecast_dates})
forecast_set['Date'] = pd.to_datetime(forecast_set['Date'])
forecast_set

In [ ]:
forecast_dates.shape

In [ ]:
stacked_np.dtype

In [ ]:
np.save('stacked_np.npy', stacked_np)

In [ ]:
shifted_df_as_np_flipped = np.fliplr(stacked_np)
shifted_df_as_np_flipped

In [ ]:
shifted_df_as_tensor = torch.tensor(shifted_df_as_np_flipped.copy())
torch.set_printoptions(precision=6)
last = shifted_df_as_tensor[-1][1:]
last = last.unsqueeze(0).unsqueeze(-1)
last.shape

In [ ]:
shifted_df_as_tensor.shape

In [ ]:
shifted_df

In [ ]:
shifted_df.to_csv('shifted_df.csv', index=False, float_format='%.5f')

In [ ]:
shifted_df_no_date = shifted_df.drop(['Date'], axis=1)

In [ ]:
import joblib
joblib.dump(prices_scaler, 'prices_scaler.pkl')

In [ ]:
for i in range(forecast_dates.shape[0]):
    last = shifted_df_as_tensor[-1][1:]
    last = last.unsqueeze(0).unsqueeze(-1).float()
    with torch.no_grad():
        predicted = model(last.to(device)).to('cpu').numpy()
    row = torch.cat([shifted_df_as_tensor[-1][1:],torch.tensor(predicted).reshape(1)],dim=0).unsqueeze(0)
    quick_reverse = prices_scaler.inverse_transform(row[:,4:].to('cpu').numpy())
    new_row = {'Price':quick_reverse[0][7], 
               'Price(t-1)':quick_reverse[0][6],
               'Price(t-2)':quick_reverse[0][5],
               'Price(t-3)':quick_reverse[0][4],
               'Price(t-4)':quick_reverse[0][3],
               'Price(t-5)':quick_reverse[0][2],
               'Price(t-6)':quick_reverse[0][1],
               'Price(t-7)':quick_reverse[0][0],
               'Real_GDP':df.iloc[-1]["Real_GDP"],
               'CPI':df.iloc[-1]["CPI"],
               'inflation_rate':df.iloc[-1]["inflation_rate"],
               'PALLFNFINDEXM':df.iloc[-1]["PALLFNFINDEXM"]
               }
    shifted_df_as_tensor = torch.cat((shifted_df_as_tensor, row),dim=0)
    shifted_df_no_date = shifted_df_no_date._append(new_row,ignore_index=True)
    print(i)
print(shifted_df_no_date) 
    

In [ ]:
df

In [ ]:
shifted_df_as_tensor.shape

In [ ]:
final_set = prices_scaler.inverse_transform(shifted_df_as_tensor[:,4:].to('cpu').numpy())
final_set

In [ ]:
shifted_df_no_date

In [ ]:
display_df = pd.DataFrame(shifted_df_no_date["Price"])
display_df

In [ ]:
display_df = display_df.iloc[6049:]
display_df

In [ ]:
display_df = display_df.reset_index(drop=True)

In [ ]:
display_df

In [ ]:
forecast_set = pd.concat([forecast_set, display_df],axis=1)

In [ ]:
forecast_set